In [3]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

# Custom Loss Function

In [62]:
def huber_fn(y_true,y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error)<1
    squared_loss = tf.square(error)/2
    linear_loss = tf.abs(error)-.5
    return tf.where(is_small_error,squared_loss,linear_loss)
def create_huber(threshold=1.0):
    def huber_fn(y_true,y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error)<threshold
        squared_loss = tf.square(error)/2
        linear_loss = threshold * tf.abs(error) - threshold**2/2
        return tf.where(is_small_error,squared_loss,linear_loss)
    return huber_fn

In [63]:
X_train[1,:].shape

(8,)

In [64]:
input_shape = X_train[1,:].shape
model = keras.models.Sequential([
    keras.layers.Dense(30,activation = "selu", kernel_initializer = "lecun_normal",
                      input_shape=input_shape),
    keras.layers.Dense(1)
])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 30)                270       
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [65]:
model.compile(loss=huber_fn,optimizer="nadam",metrics=["mae"])
model.fit(X_train_scaled,y_train,epochs=10,validation_data=(X_valid_scaled,y_valid))

Epoch 1/10
363/363 [==============================] - 1s 1ms/step - loss: 0.5986 - mae: 0.9701 - val_loss: 0.2257 - val_mae: 0.5257
Epoch 2/10
363/363 [==============================] - 0s 1ms/step - loss: 0.2159 - mae: 0.5134 - val_loss: 0.1981 - val_mae: 0.4805
Epoch 3/10
363/363 [==============================] - 0s 1ms/step - loss: 0.2033 - mae: 0.4949 - val_loss: 0.2073 - val_mae: 0.4867
Epoch 4/10
363/363 [==============================] - 0s 964us/step - loss: 0.1987 - mae: 0.4878 - val_loss: 0.2038 - val_mae: 0.4858
Epoch 5/10
363/363 [==============================] - 0s 990us/step - loss: 0.1946 - mae: 0.4809 - val_loss: 0.1881 - val_mae: 0.4636
Epoch 6/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1913 - mae: 0.4755 - val_loss: 0.1866 - val_mae: 0.4653
Epoch 7/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1881 - mae: 0.4707 - val_loss: 0.1809 - val_mae: 0.4516
Epoch 8/10
363/363 [==============================] - 0s 1ms/step - loss

In [66]:
model.save("../Models/5_CustomModelAndTraining")

INFO:tensorflow:Assets written to: ../Models/5_CustomModelAndTraining/assets


In [67]:
model = keras.models.load_model("../Models/5_CustomModelAndTraining",
                                custom_objects={"huber_fn":huber_fn})

In [68]:
model.fit(X_train_scaled,y_train,epochs=10,validation_data=(X_valid_scaled,y_valid))

Epoch 1/10
363/363 [==============================] - 1s 2ms/step - loss: 0.1774 - mean_absolute_error: 0.4537 - val_loss: 0.1766 - val_mean_absolute_error: 0.4487
Epoch 2/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1754 - mean_absolute_error: 0.4498 - val_loss: 0.1620 - val_mean_absolute_error: 0.4305
Epoch 3/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1733 - mean_absolute_error: 0.4463 - val_loss: 0.1715 - val_mean_absolute_error: 0.4389
Epoch 4/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1718 - mean_absolute_error: 0.4432 - val_loss: 0.1710 - val_mean_absolute_error: 0.4389
Epoch 5/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1699 - mean_absolute_error: 0.4408 - val_loss: 0.1849 - val_mean_absolute_error: 0.4484
Epoch 6/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1691 - mean_absolute_error: 0.4385 - val_loss: 0.1713 - val_mean_absolute_error: 0.4413
Epoch 7/10
363/3

# Custom Loss Class
### Threshold is saved with saving of model

In [69]:
class HuberLoss(keras.losses.Loss):
    def __init__(self,threshold = 1.0,**kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self,y_true,y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error)<self.threshold
        squared_loss = tf.square(error)/2
        linear_loss = self.threshold* tf.abs(error)-self.threshold**2/2
        return tf.where(is_small_error,squared_loss,linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return{**base_config,"threshold":self.threshold}

In [70]:
model.compile(loss=HuberLoss(2.0),optimizer="nadam",metrics=["mae"])
model.save("../Models/5_CustomModelAndTraining_HuberClass")

INFO:tensorflow:Assets written to: ../Models/5_CustomModelAndTraining_HuberClass/assets


In [71]:
model = keras.models.load_model("../Models/5_CustomModelAndTraining_HuberClass/",
                               custom_objects={"HuberLoss":HuberLoss})

In [72]:
model.fit(X_train_scaled,y_train,epochs=10,validation_data=(X_valid_scaled,y_valid))

Epoch 1/10
363/363 [==============================] - 1s 1ms/step - loss: 0.1833 - mean_absolute_error: 0.4325 - val_loss: 0.2052 - val_mean_absolute_error: 0.4372
Epoch 2/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1827 - mean_absolute_error: 0.4307 - val_loss: 0.1921 - val_mean_absolute_error: 0.4347
Epoch 3/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1811 - mean_absolute_error: 0.4302 - val_loss: 0.2291 - val_mean_absolute_error: 0.4511
Epoch 4/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1817 - mean_absolute_error: 0.4300 - val_loss: 0.1744 - val_mean_absolute_error: 0.4205
Epoch 5/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1806 - mean_absolute_error: 0.4290 - val_loss: 0.2091 - val_mean_absolute_error: 0.4385
Epoch 6/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1798 - mean_absolute_error: 0.4272 - val_loss: 0.1943 - val_mean_absolute_error: 0.4303
Epoch 7/10
363/3

## Custom Activation, Initializer,Reularizer Functions

In [73]:
def my_softplus(z):
    return tf.math.log(tf.exp(z)+1.0)
def my_glorot_initializer(shape,dtype=tf.float32):
    stddev = tf.sqrt(2./(shape[0]+shape[1]))
    return tf.random.normal(shape,stddev=stddev,dtype=dtype)
def my_li_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01*weights))
def my_positive_weights(weights):
    return tf.where(weights<0.,tf.zeros_like(weights),weights)

In [74]:
layer = keras.layers.Dense(30,activation = my_softplus,
                          kernel_initializer = my_glorot_initializer,
                          kernel_regularizer=my_li_regularizer,
                          kernel_constraint = my_positive_weights)

In [75]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self,factor):
        self.factor = factor
    def __call__(self,weights):
        return tf.reduce_sum(tf.abs(self.factor*weights))
    def get_config(self):
        return {"factor":factor}

In [76]:
class HuberMetric(keras.metrics.Metric):
    def __init__(self,threshold=1.0,**kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total",initializer="zeros")
        self.count = self.add_weight("count",initializer="zeros")
    def update_state(self,y_true,y_pred,sample_weight=None):
        metric = self.huber_fn(y_true,y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true),tf.float32))
    def result(self):
        return self.total/self.count
    def get_config(self):
        base_config = super().get_config()
        return{**base_config,"threshold":threshold}

In [77]:
#Better Implementation
class HuberMetric(keras.metrics.Mean):
    def __init__(self, threshold=1.0, name='HuberMetric', dtype=None):
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        super().__init__(name=name, dtype=dtype)
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        super(HuberMetric, self).update_state(metric, sample_weight)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [78]:
model.compile(loss=keras.losses.Huber(2.0), optimizer="nadam", weighted_metrics=[HuberMetric(2.0)])


In [84]:
sample_weight = np.random.rand(len(y_train))
history = model.fit(X_train_scaled.astype(np.float32), y_train.astype(np.float32),
                    epochs=2, sample_weight=sample_weight)

Epoch 1/2
363/363 [==============================] - 0s 963us/step - loss: 0.0876 - HuberMetric: 0.1753
Epoch 2/2
363/363 [==============================] - 0s 937us/step - loss: 0.0872 - HuberMetric: 0.1745


In [80]:
history.history["loss"][0], history.history["HuberMetric"][0] * sample_weight.mean()


(0.08776029944419861, 0.08776026960547038)